# L1 spectrum value check

> Check that all L1 spectra have valid flux/ivar/sensfunc arrays.

In [ ]:
# |default_exp diagnostics.l1_spectrum_value_check

In [ ]:
# |export

import matplotlib.pyplot as plt
import numpy as np

from qagmire.data import (
    get_lr_l1_single_files,
    read_l1_data,
)
from qagmire.quality_assurance import Diagnostics

To write checks of the data, we create a subclass of `Diagnostics` and implement the `tests` method.

In [ ]:
# |export


class L1SpectrumValueCheck(Diagnostics):
    """L1 spectrum value check.

    A reproduction of the class with the same name in the weaveio
    [value_checks](https://github.com/bamford/QAG/blob/master/diagnostics/value_checks.py).

    This tests for the following cases:

    * Are there non-finite pixel values?

    for the red and blue 'flux', 'flux_noss', 'ivar', 'ivar_noss' and 'sensfunc' arrays, as well as:

    * Are there negative pixel values?

    for the red and blue 'ivar', 'ivar_noss' and 'sensfunc' arrays.
    """

    def __init__(
        self,
        n_allowed_bad: int = 0,  # the number of allowed bad pixels per spectrum
        camera=None,  # limit to a specific camera: RED or BLUE
    ):
        self.n_allowed_bad = n_allowed_bad
        self.camera = camera.upper()
        super().__init__()

    def tests(self, **kwargs):
        files = get_lr_l1_single_files(**kwargs)
        data = read_l1_data(files)

        if self.camera is not None:
            camera_match = data["CAMERA"] == self.camera
            data = data.sel(filename=camera_match)
            cameras = [self.camera]
        else:
            cameras = ["RED", "BLUE"]

        # perform the tests by RUN, rather than filename
        data = data.swap_dims(filename="RUN")

        neg = data < 0
        nan = ~np.isfinite(data)

        any_neg = neg.any(dim=["LAMBDA_R", "LAMBDA_B"])
        any_nan = nan.any(dim=["LAMBDA_R", "LAMBDA_B"])

        tests = []
        for camera in cameras:
            camera_match = data["CAMERA"] == camera
            for ext in ["FLUX", "FLUX_NOSS", "IVAR", "IVAR_NOSS", "SENSFUNC"]:
                name = f"{camera}_{ext}"
                tests.append(
                    {
                        "name": f"nans_in_{name}",
                        "description": f"Are there non-finite values in {name}?",
                        "test": any_nan[name] & camera_match,
                    }
                )
            for ext in ["IVAR", "IVAR_NOSS", "SENSFUNC"]:
                name = f"{camera}_{ext}"
                tests.append(
                    {
                        "name": f"negs_in_{name}",
                        "description": f"Are there negative values in {name}?",
                        "test": any_neg[name] & camera_match,
                    }
                )
        return tests

## Tests

We use multiple `dask` workers to speed up this test.

In [ ]:
from dask.distributed import Client

client = Client(n_workers=8, threads_per_worker=1, memory_limit="2GiB")

/home2/bamford/anaconda/envs/qagmire/lib/python3.12/contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (4.07s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
Task exception was never retrieved
future: <Task finished name='Task-9774' coro=<Client._gather.<locals>.wait() done, defined at /home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/client.py:2208> exception=AllExit()>
Traceback (most recent call last):
  File "/home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/client.py", line 2217, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-9764' coro=<Client._gather.<locals>.wait() done, defined at /home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/client.py:2208> exc

### Red camera

In [ ]:
tests = L1SpectrumValueCheck(camera="RED")
tests.run(date="2017*")

Locating and converting where necessary: 100%|██████████| 60/60 [00:00<00:00, 1783.41it/s]
Reading netCDF files... took 6.88 s. Size is 27397.908 Mb


2024-02-28 19:22:39,476 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('concatenate-open_dataset-getitem-b2f00c785c2ff18829eeaace2f95846c', 10, 0, 0))" coro=<Worker.execute() done, defined at /home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2024-02-28 19:22:39,483 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('concatenate-where-getitem-3cd475536cc7f23557465c85048c4cff', 23, 0, 0))" coro=<Worker.execute() done, defined at /home2/bamford/anaconda/envs/qagmire/lib/python3.12/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2024-02-28 19:22:39,483 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('where-getitem-9f75473910392f2ba3ace49952448c93', 8, 0, 0))" coro=<Worker.execute() done, defined at /home2/bamford/anacond

KeyboardInterrupt: 

In [ ]:
tests.summary(by="RUN")

In [ ]:
tests.summary(by="NSPEC")

### Blue camera

In [ ]:
tests = L1SpectrumValueCheck(camera="BLUE")
tests.run(date="2017*")

In [ ]:
tests.summary(by="RUN")

In [ ]:
tests.summary(by="NSPEC")

## Verification

In [ ]:
files = get_lr_l1_single_files(date="2017*")
data = read_l1_data(files)
data = data.swap_dims(filename="RUN")

In [ ]:
def plot(run, ext):
    qty = data.sel(RUN=run)[ext]
    fig, ax = plt.subplots(3, 1, figsize=(15, 5), sharex=True, sharey=True)
    ax[0].imshow(qty, vmin=0, vmax=100, interpolation="none")
    ax[0].set_title(ext)
    ax[1].imshow(~np.isfinite(qty), interpolation="none")
    ax[1].set_title("NaNs")
    ax[2].imshow(qty < 0, interpolation="none")
    ax[2].set_title("Negs")
    fig.suptitle(f"RUN {run}")

In [ ]:
plot(1004097, "RED_FLUX")

In [ ]:
plot(1004098, "BLUE_FLUX")

In [ ]:
plot(1003329, "RED_FLUX")

In [ ]:
plot(1003330, "BLUE_FLUX")

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()